Make the disability block accessibility

The count rated sidewalks are stored by previous codes.

In [4]:
import json
import geojson
from shapely.geometry import shape, Point, Polygon
import geopandas as gpd
import folium


complete_gdf = gpd.read_file('../data/compelete_scores.json')
# sidewalks_gdf = gpd.read_file('../data/Sidewalks.json')
result_gdf = complete_gdf

In [43]:
new_gdf = result_gdf
# Create a map centered at a specific location
m = folium.Map(location=[42.361145, -71.057083], zoom_start=30)

# Add sidewalks layer to the map with color ramp based on average score
folium.Choropleth(
    geo_data=new_gdf,
    name='Average Score',
    data=new_gdf,
    columns=['SWK_ID', 'count'],
    key_on='properties.SWK_ID',
    fill_color='RdYlBu_r',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Average Score',
    highlight=True,
    show=True,
    bins=[0, 2, 4, 5, 6, 20]
).add_to(m)

#Why there are no geometry data
print(new_gdf['count'].isna().sum())
new_gdf = new_gdf.dropna(subset=['geometry'])

# Adds the ability to click the sidewalk to see the score for that sidewalk
folium.GeoJson(
    new_gdf,
    name='Sidewalk Scores',
    tooltip=folium.features.GeoJsonTooltip(fields=['SWK_ID','count'], labels=True, sticky=True),
    style_function=lambda feature: {'weight': 1, 'color':'black', 'fillOpacity':0},
    popup=folium.features.GeoJsonPopup(fields=['SWK_ID','count'], labels=True, sticky=True),
    show=True
).add_to(m)


# Add a layer control to the map
# folium.LayerControl().add_to(m)

# Save the map as an HTML file
# m.save('maps/counted_sidewalk_map.html')

0


Deal with the data and then use sjoin to combine

In [27]:
neighborhoods_gdf = gpd.read_file('../data/Climate_Ready_Boston_Social_Vulnerability.geojson')
another_gdf = gpd.read_file('../data/compelete_scores.json')
sidewalk_gdf = another_gdf.to_crs(neighborhoods_gdf.crs)

In [30]:
neighborhoods_gdf.columns

Index(['FID', 'GEOID10', 'AREA_SQFT', 'AREA_ACRES', 'POP100_RE', 'HU100_RE',
       'TotDis', 'TotChild', 'OlderAdult', 'Low_to_No', 'LEP', 'POC2',
       'MedIllnes', 'Name', 'Shape__Area', 'Shape__Length', 'geometry'],
      dtype='object')

In [33]:
joined_gdf = gpd.sjoin(sidewalk_gdf, neighborhoods_gdf, how='left', op='within')
neighborhood_scores = joined_gdf.groupby('FID')['count'].mean().reset_index()
neighborhoods_with_scores_gdf = neighborhoods_gdf.merge(neighborhood_scores, on='FID', how='left')
# neighborhoods_with_scores_gdf = neighborhoods_with_scores_gdf.dropna(subset=['count'])
neighborhoods_with_scores_gdf['total_risk'] = (neighborhoods_with_scores_gdf['TotDis'] / (neighborhoods_with_scores_gdf['AREA_SQFT']/1000000))* neighborhoods_with_scores_gdf['count']
print(neighborhoods_with_scores_gdf[['TotDis','count','AREA_SQFT','total_risk']])

c:\Users\xiao\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3382: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


     TotDis     count   AREA_SQFT  total_risk
0       470  1.121212  3914567.54  134.617602
1       299  1.117647  1472713.92  226.912007
2        84  1.093023  1376667.12   66.692922
3        45  1.317460  3228780.12   18.361645
4       131  2.100000  2741497.18  100.346629
..      ...       ...         ...         ...
175     329  1.230088  4024571.84  100.557061
176     715  1.184874  6819156.25  124.236026
177     340  1.400000  3619583.13  131.506857
178     268  1.508961  6678367.07   60.553939
179    1146  1.208333  6246271.61  221.692249

[180 rows x 4 columns]


Add one layer of the total risk of disabilities. Another layer of original condition of the block

In [44]:
# Add sidewalks layer to the map with color ramp based on average score

folium.Choropleth(
    geo_data=neighborhoods_with_scores_gdf,
    name='Neighborhood Colors',
    data=neighborhoods_with_scores_gdf,
    columns=['FID', 'total_risk'],
    key_on='feature.properties.FID',
    fill_color='Reds',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Average Score',
    highlight=True,
    show=False
).add_to(m)

print(neighborhoods_with_scores_gdf['total_risk'].isna().sum())


# Adds the ability to click the sidewalk to see the score for that sidewalk
folium.GeoJson(
    neighborhoods_with_scores_gdf,
    name='Neighborhood Scores',
    tooltip=folium.features.GeoJsonTooltip(fields=['FID'], labels=True, sticky=True),
    style_function=lambda feature: {'weight': 1, 'color':'black', 'fillOpacity':0},
    popup=folium.features.GeoJsonPopup(fields=[ 'Name','total_risk','TotDis','count'], labels=True, sticky=True),
    show=False
).add_to(m)




# Add a layer control to the map
# folium.LayerControl().add_to(m)

# Save the map as an HTML file
# m.save('maps/disability_scores_hazards.html')

2


In [45]:
# Add sidewalks layer to the map with color ramp based on average score
# m = folium.Map(location=[42.361145, -71.057083], zoom_start=30)

folium.Choropleth(
    geo_data=neighborhoods_with_scores_gdf,
    name='Hazards Colors',
    data=neighborhoods_with_scores_gdf,
    columns=['FID', 'count'],
    key_on='feature.properties.FID',
    fill_color='Blues',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Average Count',
    highlight=True,
    show=False
).add_to(m)

print(neighborhoods_with_scores_gdf['total_risk'].isna().sum())


# Adds the ability to click the sidewalk to see the score for that sidewalk
folium.GeoJson(
    neighborhoods_with_scores_gdf,
    name='Hazards Scores',
    tooltip=folium.features.GeoJsonTooltip(fields=['FID'], labels=True, sticky=True),
    style_function=lambda feature: {'weight': 1, 'color':'black', 'fillOpacity':0},
    popup=folium.features.GeoJsonPopup(fields=[ 'Name','count'], labels=True, sticky=True),
    show=False
).add_to(m)




# Add a layer control to the map
folium.LayerControl().add_to(m)

# Save the map as an HTML file
m.save('maps/disability_scores_hazards.html')

2
